In [65]:
import os 
import numpy as np
from sent_sampling.utils.data_utils import load_obj, SAVE_DIR, UD_PARENT, RESULTS_DIR, LEX_PATH_SET, save_obj, ANALYZE_DIR
from sent_sampling.utils import extract_pool
from sent_sampling.utils.optim_utils import optim_pool
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn
from tqdm import tqdm 
import pandas as pd 
import pickle
import xarray as xr 
import ftfy
from scipy.spatial.distance import pdist, squareform
import scipy
import copy 

In [8]:
RESULTS_DIR=os.path.join(SAVE_DIR,'results')
RESULTS_DIR

'/om/user/ehoseini/MyData/sent_sampling/results'

# Read Ev's file 

In [6]:
file_name='U01_sentselection_Dec18-2020_updDec23.xlsx'
df_ev_selected=pd.read_excel(os.path.join(RESULTS_DIR,f"{file_name}"))

In [7]:
ev_sentences=df_ev_selected.sentence[df_ev_selected.previously_selected_by_ev==1]
extractor_name='group=best_performing_pereira_1-dataset=ud_sentencez_token_filter_v3-activation-bench=None-ave=False'
ext_obj=extract_pool[extractor_name]()
ext_obj.load_dataset()
ext_obj()

934it [00:00, 207790.80it/s]
934it [00:00, 203770.09it/s]
935it [00:00, 206089.35it/s]
934it [00:00, 207812.84it/s]
934it [00:00, 203420.91it/s]
935it [00:00, 170856.72it/s]
934it [00:00, 208865.43it/s]
934it [00:00, 203241.50it/s]
935it [00:00, 177910.19it/s]


extracting network activations for roberta-base
ud_sentencez_token_filter_v3_roberta-base_layer_2_activation_ave_False.pkl already exists, loading...
loading /om/user/ehoseini/MyData/sent_sampling/ud_sentencez_token_filter_v3_roberta-base_layer_2_activation_ave_False.pkl
adding activations to set
extracting network activations for xlnet-large-cased
ud_sentencez_token_filter_v3_xlnet-large-cased_layer_24_activation_ave_False.pkl already exists, loading...
loading /om/user/ehoseini/MyData/sent_sampling/ud_sentencez_token_filter_v3_xlnet-large-cased_layer_24_activation_ave_False.pkl
adding activations to set
extracting network activations for bert-large-uncased-whole-word-masking
ud_sentencez_token_filter_v3_bert-large-uncased-whole-word-masking_layer_12_activation_ave_False.pkl already exists, loading...
loading /om/user/ehoseini/MyData/sent_sampling/ud_sentencez_token_filter_v3_bert-large-uncased-whole-word-masking_layer_12_activation_ave_False.pkl
adding activations to set
extracting n

In [10]:
optimizer_obj = optim_pool['coordinate_ascent-obj=D_s-n_iter=2000-n_samples=300-n_init=2-run_gpu=False']()
optimizer_obj.load_extractor(ext_obj)

In [11]:
sentences=[x['text'] for x in ext_obj.data_]
ev_sent_id=([sentences.index(ev_sent) for ev_sent in ev_sentences])

In [12]:
sentence_keys=[x['word_FORM'] for x in ext_obj.data_]
ev_sentence_keys=[sentence_keys[x] for x in ev_sent_id]


In [64]:
# get activations for ev sentences 
model_name_list=[]
activation_list=[]
for act in optimizer_obj.activations:
    model_name_list.append(act['model_name'])
    activation=act['activations']
    activation_list.append(np.stack([activation[x] for x in ev_sent_id]))

/usr/local/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [14]:
with open('/om/user/ehoseini/MyData/ecog_DNN/crunched/EM1049_ecog_ANNSET1_auditory_stim_resp_struct.pkl','rb') as fid:
    sub_dat=pickle.load(fid)

In [15]:
# select significant electrodes 
elec_data_dec=sub_dat.elec_data_dec
sig_elecs=sub_dat.elec_data_zs_dec_s_vs_n.isin(1).values
good_elecs=elec_data_dec.elec_ch_label[sig_elecs]
sig_electrodes=elec_data_dec.sel(elec_ch_label=good_elecs)

In [16]:
# select last word 
grouped=sig_electrodes.groupby(sig_electrodes.sentence_id,squeeze=True)
group_set=[]
for name, group in grouped:
    group_set.append(group[-1,:])
elec_data_dec_last=xr.concat(group_set)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: the `dim` argument to `concat` will be required in a future version of xarray; for now, setting it to the old default of 'concat_dim'
  


In [17]:
correspondance=[]
loc_corres=[]
for x in np.unique(elec_data_dec.sentence_id.values):
    ex=elec_data_dec.where(elec_data_dec.sentence_id==x,drop=True).string.values.tolist()
    xs=[]
    for sent_keys in ev_sentence_keys:
        x=np.asarray([elem in sent_keys for elem in ex])
        xs.append(x.sum())
    correspondance.append([' '.join(ex),
        ev_sentences.tolist()[np.asarray(xs).argmax()]])
    loc_corres.append(np.asarray(xs).argmax())

In [18]:
# reorder the data based on evsentences: 
elec_data_dec_last.string[10]


<xarray.DataArray 'string' ()>
array('budget', dtype='<U16')
Coordinates:
    sentence_id  int64 11
    key          <U7 'word_10'
    string       <U16 'budget'

In [19]:
mismatch=np.where(np.asarray([ev_sentence_keys[x][-1] for x in loc_corres])!=elec_data_dec_last.string.values)[0]
mismatch

array([102, 107, 129, 139])

In [20]:
elec_data_dec_last

<xarray.DataArray 'elec_data_dec' (concat_dims: 200, elec_ch_label: 43)>
array([[1.050939, 0.316552, 0.53669 , ..., 0.327903, 0.421479, 0.472974],
       [0.808003, 0.32005 , 0.587307, ..., 0.36979 , 0.54108 , 0.563186],
       [0.980192, 0.367036, 0.60057 , ..., 0.350946, 0.500173, 0.398446],
       ...,
       [0.968806, 0.368922, 0.475369, ..., 0.315522, 0.536447, 0.391728],
       [0.85131 , 0.304012, 0.547084, ..., 0.40276 , 0.555275, 0.439646],
       [0.835805, 0.39997 , 0.660798, ..., 0.367074, 0.525117, 0.39376 ]])
Coordinates:
  * elec_ch_label  (elec_ch_label) object 'LFIa1' 'LFIb6' ... 'LTP9' 'LTP10'
    sentence_id    (concat_dims) int64 1 2 3 4 5 6 7 ... 195 196 197 198 199 200
    key            (concat_dims) <U7 'word_11' 'word_13' ... 'word_6' 'word_14'
    string         (concat_dims) <U16 'face' 'afford' ... 'far' 'investigated'
Dimensions without coordinates: concat_dims

In [21]:
ev_sentence_reorder=[' '.join(ev_sentence_keys[x]) for x in loc_corres]
ev_sentence_reorder[139]

"Her left arm looked like the front window of Tiffany 's"

In [22]:
elec_data_dec[mismatch[0],:]

<xarray.DataArray 'elec_data_dec' (elec_ch_label: 126)>
array([0.848429, 0.765663, 0.339164, 0.316129, 0.32603 , 0.33419 , 0.282434,
       0.279201, 0.423018, 0.33317 , 0.300991, 0.599692, 0.54112 , 0.694383,
       0.458684, 0.679506, 0.417404, 0.439016, 0.47244 , 0.576683, 0.302358,
       0.353904, 0.572206, 0.672607, 0.424037, 0.509382, 0.342481, 0.768057,
       0.382584, 0.443446, 0.464365, 0.45949 , 1.352317, 1.683111, 1.394609,
       0.756948, 0.74048 , 0.735696, 0.698315, 1.234574, 0.95204 , 1.028133,
       0.704788, 0.759211, 1.391191, 1.323002, 0.777524, 0.400619, 0.338611,
       0.278375, 0.287862, 0.303731, 0.302842, 0.272513, 0.323725, 0.364867,
       0.319672, 0.410871, 0.604126, 0.751009, 0.646955, 0.698013, 0.798032,
       0.499603, 0.365257, 0.201945, 0.204811, 0.20619 , 0.18668 , 0.204885,
       0.269033, 0.298178, 0.441538, 0.381184, 1.573895, 2.438745, 0.70902 ,
       0.462748, 0.414119, 0.376987, 0.368363, 0.443855, 0.511173, 0.530419,
       0.430174, 0.6

In [20]:
mismatch

array([102, 107, 129, 139])

In [23]:
# select last word 
grouped=sig_electrodes.groupby(sig_electrodes.sentence_id,squeeze=True)
sentence_set=[]
for name, group in grouped:
    sentence_set.append(' '.join(group.string.values))


In [26]:
activation_reorder=[x[loc_corres,:] for x in activation_list]

In [27]:
activaton_fix=[]
for x in activation_reorder: 
    new_act=[]
    for y in x:
        if len(y)==3:
            y=y[0]
            
        else:
            y=y
        new_act.append(y)
    activaton_fix.append(np.stack(new_act))

In [28]:


model_corr=[pdist(x,'correlation') for x in activaton_fix]

In [29]:
test=pd.DataFrame(np.transpose(elec_data_dec_last.values))
sub_corr=np.asarray(test.corr())

In [30]:
sub_corrr=np.triu(sub_corr,k=1)+np.transpose(np.triu(sub_corr,k=1))

In [31]:
sub_corr_vec=squareform(sub_corrr,checks=False)


In [57]:
np.expand_dims(sub_corr_vec,axis=0).shape

(1, 19900)

In [70]:
model_plus_sub=np.concatenate((np.asarray(model_corr),np.expand_dims(sub_corr_vec,axis=0)),axis=0)
model_sub_list=copy.deepcopy(model_name_list)
model_sub_list.append('subject')



In [72]:
mode_subj_corr=pd.DataFrame(np.transpose(model_plus_sub),columns=model_sub_list)

,roberta-base,xlnet-large-cased,bert-large-uncased-whole-word-masking,xlm-mlm-en-2048,gpt2-xl,albert-xxlarge-v2,ctrl,subject
roberta-base,1.000000,-0.253057,0.406880,-0.164958,-0.061608,-0.075345,-0.186081,-0.028120
xlnet-large-cased,-0.253057,1.000000,-0.239365,-0.063039,-0.195530,0.021580,-0.205474,0.031606
bert-large-uncased-whole-word-masking,0.406880,-0.239365,1.000000,0.012942,-0.079668,-0.042384,-0.238746,-0.005927
xlm-mlm-en-2048,-0.164958,-0.063039,0.012942,1.000000,-0.007963,0.014400,-0.066774,0.061260
gpt2-xl,-0.061608,-0.195530,-0.079668,-0.007963,1.000000,-0.040437,-0.059265,-0.042409
albert-xxlarge-v2,-0.075345,0.021580,-0.042384,0.014400,-0.040437,1.000000,-0.123571,0.030001
ctrl,-0.186081,-0.205474,-0.238746,-0.066774,-0.059265,-0.123571,1.000000,0.012159
subject,-0.028120,0.031606,-0.005927,0.061260,-0.042409,0.030001,0.012159,1.000000
